In [2]:
import sys
import pymol
from pymol import cmd, stored

In [3]:
pymol.pymol_argv = ['pymol', '-qc'] # quiet / no GUI
stdout = sys.stdout
stderr = sys.stderr
pymol.finish_launching(['pymol', '-xiq'])
sys.stdout = stdout
sys.stderr = stderr

In [4]:
import numpy as np
import os
from matplotlib import pyplot as plt
%run 'functions_dump.py'

/Users/karen/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/karen/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [5]:
files_dump_folder = '/Users/karen/HIS3InterspeciesEpistasis/Analysis/Karen/files_dump/'
aa3 = "ALA CYS ASP GLU PHE GLY HIS ILE LYS LEU MET ASN PRO GLN ARG SER THR VAL TRP TYR XXX".split()
ascii_letters_upper = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

### PyMoling

In [6]:
swiss_mono_file = os.path.join(files_dump_folder, 'structure_predictions', 'HIS3_saccharomyces_cerevisiae__swissmodel_prediction__mono.pdb')
assembly_4lom_file = os.path.join(files_dump_folder, 'structure_visualizations', '4lom_assembly.pdb')
temp_file = '/Users/karen/Downloads/ptest2.pdb'

cmd.reinitialize()
cmd.load(assembly_4lom_file, '4lom_assembly')

# loading swiss monomers 24 times
for i in range(1,25):
    cmd.do("load %s, swiss-%s" % (swiss_mono_file, i))

# aligning swiss monomers to 4lom assembly
for i in range(1,25):
    cmd.do("align swiss-%s, 4lom_assembly, target_state=%s"  %(i, i))
    
# renaming chains because currently there is only chainA 
for i in range(1,25):
    cmd.do("alter swiss-%s, chain='%s'" %(i, ascii_letters_upper[i-1]))
    
merger = 'or'.join([" swiss-%s " % i for i in range(1,25)])
save_session('/Users/karen/Downloads/4lom_and_his3.pse')
cmd.do('save %s, %s' %(temp_file, merger))

# save_session('/Users/karen/Downloads/swiss_assembly.pse')

 Applying pse_export_version=1.720 compatibility
 Save: wrote "/Users/karen/Downloads/ptest2.pdb".


In [ ]:
from IPython.html.widgets.widget_float import FloatProgress

In [21]:
import itertools

cmd.reinitialize()
cmd.load(temp_file, 'swiss_assembly')

interfaces = {}
f = FloatProgress(min=0, max=len(list(itertools.combinations(range(24), 2))))
display(f)
for chain_index1, chain_index2 in (itertools.combinations(range(24), 2)):
    chain1, chain2 = ascii_letters_upper[chain_index1], ascii_letters_upper[chain_index2]
    returned = interfaceResidues('swiss_assembly', cA = 'c. %s' %chain1, cB = 'c. %s' %chain2)
    interfaces[chain1, chain2] = returned
    f.value += 1

In [23]:
interface_residues = []
for k,v in interfaces.items():
    interface_residues.extend([e[1] for e in v])

In [29]:
interface_residues = set([int(p) for p in interface_residues])

In [26]:
cmd.reinitialize()
open_or_fetch(temp_file)
white_and_beautiful()
color_positions(set(interface_residues))
save_session('/Users/karen/Downloads/test123.pse')

 Applying pse_export_version=1.720 compatibility


## Lucas

In [27]:
position_translation = pd.read_table('/Users/karen/Downloads/position_translation.csv')
position_translation.set_index('relative_position', inplace=True)

def get_absolute_position(segment_number, relative_position):
    return int(position_translation.iloc[relative_position]['S'+str(segment_number)])

lucas_sign_epistasis = pd.read_excel('/Users/karen/Downloads/SignEpiPairs.xlsx')
lucas_sign_epistasis['VarPos_absolute'] = lucas_sign_epistasis.apply(lambda df: get_absolute_position(df['SegN'], df['VarPos']-1), axis=1)
lucas_sign_epistasis['SubPos_absolute'] = lucas_sign_epistasis.apply(lambda df: get_absolute_position(df['SegN'], df['SubPos']-1), axis=1)
# lucas_sign_epistasis['distance'] = lucas_sign_epistasis.apply(lambda df: calc_residue_dist(residues[df['VarPos_absolute']-1], residues[df['SubPos_absolute']-1]), axis=1)

logodds_threshold = 5
lucas_sign_epistasis['logodds_thresholded'] = lucas_sign_epistasis['logodds'].apply(lambda f: min(f, logodds_threshold))

In [50]:
lucas_sign_epistasis['SubPos_interface'] = lucas_sign_epistasis['SubPos_absolute'].apply(lambda p: p in interface_residues)
lucas_sign_epistasis['VarPos_interface'] = lucas_sign_epistasis['VarPos_absolute'].apply(lambda p: p in interface_residues)
lucas_sign_epistasis['interface'] = lucas_sign_epistasis['VarPos_interface']# + lucas_sign_epistasis['SubPos_interface']

In [65]:
cmd.reinitialize()
open_or_fetch('/Users/karen/Downloads/swiss_assembly.pse')
white_and_beautiful()

sign_epistasis_interface = lucas_sign_epistasis[(lucas_sign_epistasis['interface']) & (lucas_sign_epistasis['pBon'] < 0.01)].VarPos_absolute.values
sign_epistasis_not_in_interface = lucas_sign_epistasis[(~lucas_sign_epistasis['interface']) & (lucas_sign_epistasis['pBon'] < 0.01)].VarPos_absolute.values
color_positions(set(sign_epistasis_interface), representation='spheres', constant_color='#00ff00')
color_positions(set(sign_epistasis_not_in_interface), representation='spheres', constant_color='#ff0000')
save_session('/Users/karen/Downloads/position128.pse')

 Applying pse_export_version=1.720 compatibility
